# BART API Queries

In [336]:
# Imports
import pandas as pd
from pandas.io.json import json_normalize
import datetime as dt
import dateutil.parser as dp
import numpy as np
import requests
import json
from flatten_json import flatten_json
from matplotlib import pyplot as plt

from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR,FLOAT,INTEGER,BIGINT,BLOB,DATETIME,DATE,TIMESTAMP,TIME,BOOLEAN

In [337]:
# load credentials and create db connection
with open('credentials.json') as f:
    db_creds = json.load(f)
connect_str = 'mysql+pymysql://%s:%s@%s:%s/%s' % \
              (db_creds['username'],
              db_creds['password'],
              db_creds['host'],
              db_creds['port'],
              db_creds['database'])
engine = create_engine(connect_str, echo = False)


In [338]:
# load credentials and create db connection
with open('credentials.json') as f:
    db_creds = json.load(f)
connect_str = 'mysql+pymysql://%s:%s@%s:%s/%s' % \
              (db_creds['username'], 
               db_creds['password'],
               db_creds['host'],
               db_creds['port'],
               db_creds['database'])
engine = create_engine(connect_str, echo = False)

# Static Queries (eg stations, routes, etc.)

In [339]:
# stations
rest_url = "http://api.bart.gov/api/stn.aspx?cmd=stns&key=MW9S-E7SL-26DU-VV8V&json=y"
response = requests.post(rest_url)
response_json = json.loads(response.content)
stations = pd.io.json.json_normalize(response_json['root']['stations']['station'])

stations.to_sql('stations', engine, if_exists='replace')
stations.head()

,abbr,address,city,county,gtfs_latitude,gtfs_longitude,name,state,zipcode
0,12TH,1245 Broadway,Oakland,alameda,37.803768,-122.271450,12th St. Oakland City Center,CA,94612
1,16TH,2000 Mission Street,San Francisco,sanfrancisco,37.765062,-122.419694,16th St. Mission,CA,94110
2,19TH,1900 Broadway,Oakland,alameda,37.808350,-122.268602,19th St. Oakland,CA,94612
3,24TH,2800 Mission Street,San Francisco,sanfrancisco,37.752470,-122.418143,24th St. Mission,CA,94110
4,ANTC,1600 Slatten Ranch Road,Antioch,Contra Costa,37.995388,-121.780420,Antioch,CA,94509


In [340]:
# routes
station_info_column_names = ['abbr', 'address', 'attraction', 'city', 'county', 'cross_street',
                             'food', 'gtfs_latitude', 'gtfs_longitude', 'intro', 'link', 'name',
                             'north_platforms', 'north_routes', 'platform_info', 'shopping',
                             'south_platforms', 'south_routes', 'state', 'zipcode']
station_info_types = {'abbr': VARCHAR(4),
                      'address': VARCHAR(256),
                      'attraction': VARCHAR(512),
                      'city': VARCHAR(256),
                      'county': VARCHAR(256), 
                      'cross_street': VARCHAR(256),
                      'food': VARCHAR(512),
                      'gtfs_latitude': FLOAT, 
                      'gtfs_longitude': FLOAT,
                      'intro': VARCHAR(512),
                      'link': VARCHAR(256),
                      'name': VARCHAR(256),
                      'platform_info': VARCHAR(512),
                      'shopping': VARCHAR(256),
                      'state': VARCHAR(256),
                      'zipcode': VARCHAR(256)}

all_stations = pd.DataFrame(columns = station_info_column_names)

for i,orig in enumerate(stations['abbr']):
    try:
        rest_url = "http://api.bart.gov/api/stn.aspx?cmd=stninfo&orig=%s&key=MW9S-E7SL-26DU-VV8V&json=y" % (orig)
        response = requests.post(rest_url)
        response_json = json.loads(response.content)
        station_info = pd.io.json.json_normalize(response_json['root']['stations']['station'])
        station_info.columns = station_info_column_names

        all_stations = pd.concat([all_stations, station_info])
    except:
        print(i,orig)

all_stations.loc[:,station_info_columns].reset_index(drop=True).to_sql('station_info',engine,if_exists='replace', dtype=station_info_types)

NameError: name 'station_info_columns' is not defined

In [ ]:
c = 'north_routes'
all_stations.set_index('abbr')[c].apply(pd.Series).stack().reset_index(level=0).rename(columns={0:'route'}).reset_index(drop=True)

In [ ]:
# 
rest_url = "http://api.bart.gov/api/route.aspx?cmd=routeinfo&route=1&key=MW9S-E7SL-26DU-VV8V&json=y"
response = requests.post(rest_url)
response_json = json.loads(response.content)
trains = pd.io.json.json_normalize(response_json['root']['routes']['route'])
trains.head()

# Real Time Queries

In [ ]:
# train count
rest_url = "http://api.bart.gov/api/bsa.aspx?cmd=count&key=MW9S-E7SL-26DU-VV8V&json=y"
response = requests.post(rest_url)
response_json = json.loads(response.content)
trains = pd.io.json.json_normalize(response_json['root'])

trains['timestamp'] = pd.to_datetime(trains['date'] + ' ' + trains['time'])
train_count_dtypes = {'timestamp':DATETIME,
                      'message':VARCHAR(256),
                      'traincount':INTEGER,}

trains.loc[:,train_count_dtypes.keys()].to_sql('train_count',
                                               engine,
                                               dtype=train_count_dtypes,
                                               index=False,
                                               if_exists='append')

In [ ]:
# etd
rest_url = "http://api.bart.gov/api/etd.aspx?cmd=etd&orig=daly&key=MW9S-E7SL-26DU-VV8V&json=y"
response = requests.post(rest_url)
response_json = json.loads(response.content)
raw_etd = pd.io.json.json_normalize(response_json['root']['station'][0]['etd'])


etd = pd.DataFrame()

for i,r in raw_etd.iterrows():
    dest_etd = pd.concat([pd.io.json.json_normalize(est) for est in r['estimate']])
    for c in r.index:
        if c!='estimate':
            dest_etd[c] = r[c]
    etd = pd.concat([etd,dest_etd])
etd

# Trains Entering System

In [ ]:
# full list of pairs to check
orig_list = ['BALB','CAST','DELN','NCON','UCTY']
orig_dest_pairs = [('BALB', 'ANTC'), ('BALB', 'DUBL'), ('BALB', 'RICH'), ('BALB', 'WARM'), 
                   ('CAST', 'BAYF'), ('CAST', 'DALY'), 
                   ('DELN', 'BAYF'), ('DELN', 'MLBR'), ('DELN', 'WARM'), 
                   ('NCON', 'BAYF'), ('NCON', 'MLBR'), 
                   ('UCTY', 'DALY'), ('UCTY', 'MLBR'), ('UCTY', 'RICH'), ('UCTY', 'SFIA')]

# Empty data frame for the final list of train estimates
df_estimates = pd.DataFrame({
                                'queried_at':  [],
                                'orig':        [],
                                'dest':        [],
                                # 'origin':      [],
                                # 'destination': [],
                                'direction':   [],
                                'minutes':     [],
                                'length':      [],
                                'platform':    [],
                                'color':       [],
                                'hexcolor':    [],
                                'limited':     [],
                                'delay':       [],
                                'bikeflag':    [],
                            })
estimate_time_of_departure_dtypes = {
                                'queried_at':  TIMESTAMP,
                                'orig':        VARCHAR(4),
                                'dest':        VARCHAR(4),
                                # 'origin':      VARCHAR(64),
                                # 'destination': VARCHAR(64),
                                'direction':   VARCHAR(64),
                                'minutes':     INTEGER,
                                'length':      INTEGER,
                                'platform':    INTEGER,
                                'color':       VARCHAR(16),
                                'hexcolor':    VARCHAR(8),
                                'limited':     INTEGER,
                                'delay':       INTEGER,
                                'bikeflag':    INTEGER,
                            }

for orig in orig_list:
    print(orig, end=' ')

    rest_url = 'http://api.bart.gov/api/etd.aspx?cmd=etd&orig=%s&key=MW9S-E7SL-26DU-VV8V&json=y' % (orig)
    response = requests.post(rest_url)
    response_json = json.loads(response.content)
    timestamp_str = (response_json['root']['date'] + ' ' + response_json['root']['time'])
#     dt_queried_at = dt.datetime.strptime(timestamp_str,'%m/%d/%Y %I:%M:%S %p %Z')
    dt_queried_at = dp.parse(timestamp_str)
    print(timestamp_str)
    
    df_orig = json_normalize(response_json['root']['station'])
    try:
        df_dest = json_normalize(df_orig['etd'][0])
    except:
        continue

    # flatten json to append to df_est
    for i,d in df_dest.iterrows():
        df_this_est = json_normalize(d['estimate'])

        df_this_est['queried_at'] = dt_queried_at
        df_this_est['orig'] = df_orig.loc[0,'abbr']
        # df_this_est['origin'] = df_orig.loc[0,'name']
        df_this_est['dest'] = d['abbreviation']
        # df_this_est['destination'] = d['destination']
        df_this_est['limited'] = d['limited']

        # only keep the nearest train
        drop_idx = df_this_est.index[df_this_est['minutes'] > df_this_est['minutes'].min()]

        df_estimates = pd.concat([df_estimates, 
                                  df_this_est.drop(drop_idx)], 
                                 sort=False
                                ).reset_index(drop=True)

df_estimates.to_sql('estimate_time_of_departure',
                    engine,
                    index=False,
                    dtype=estimate_time_of_departure_dtypes,
                    if_exists='append')

BALB 07/30/2019 12:20:45 AM PDT
CAST 07/30/2019 12:20:45 AM PDT
DELN 07/30/2019 12:20:45 AM PDT
NCON 07/30/2019 12:20:45 AM PDT
UCTY 07/30/2019 12:20:45 AM PDT


In [ ]:
df_estimates